In [8]:
!pip install xmltodict
!mkdir 01_rawdata
!curl ipinfo.io/ip

mkdir: cannot create directory ‘01_rawdata’: File exists
35.230.102.204


In [0]:
import requests
import re
from datetime import datetime
from time import sleep
from math import ceil
from random import randint
import xmltodict  # Elementtree based
import json
import pandas as pd

In [0]:
#(1/3) 법령 목록 조회 및 Raw-Data 저장

today = datetime.today().strftime('%Y%m%d') # 파일명 등에 사용할 날짜 지정
directory = '01_rawdata' # 저장 위치 디렉터리명 지정

uri_OC = '<your-access-id>'  #(필수)신청 및 승인된 ID
uri_target = 'law'  #(필수)law
uri_type = 'XML'  #(필수)HTML or XML
uri_display = 100  #요청시 표출되는 법령 목록 수
uri_sort = 'lasc'  #요청시 표출되는 법령명 오름차순(기본:lasc)
uri_page = 1  #총 페이지를 조회하기 위해 지정할 최초의 페이지 번호

In [0]:
def createURIstring(OC, target, type, page, display, sort):
    uri = f'http://www.law.go.kr/DRF/lawSearch.do?OC={OC}&target={target}&type={type}&page={page}&display={display}&sort={sort}'
    return uri

In [0]:
def getTotalPageNum():
    uri = createURIstring(uri_OC, uri_target, uri_type, uri_page, uri_display, uri_sort)
    r = requests.get(uri)
    p = re.compile(r'<totalCnt>([0-9]+)</totalCnt>')
    try:
        total_item_num = int(p.findall(r.text)[0])
        total_page_num = ceil(total_item_num / uri_display)
        return total_page_num, total_item_num
    except IndexError:
        print('국가법령정보 공동활용 OPEN API의 할당 ID 및 접근가능 IP를 체크해주세요.')
    # ConnectionError: HTTPConnectionPool(host='www.law.go.kr', port=80): Max retries exceeded with url: /DRF/lawSearch.do?OC=tooget&target=law&type=XML&page=1&display=100&sort=lasc (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f4df78a2dd8>: Failed to establish a new connection: [Errno -2] Name or service not known',))

In [0]:
uri = createURIstring(uri_OC, uri_target, uri_type, uri_page, uri_display, uri_sort)
r = requests.get(uri)

In [70]:
r.text

'\ufeff<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=EUC-KR" />\n<!-- <title>[[Sorrypage#1]]</title> -->\n\n\n<title>[법제처] 국가법령정보센터</title>\n<link rel="stylesheet" type="text/css" href="/css/txt.css" />\n</head>\n\n<body leftmargin="0" topmargin="0">\n<div id="wrap">\n\t<div id="t_box">\n\t<!--\n\t\t<div class="logo"><img src="/img/logo_ncia.gif" /></div>\n\t-->\n\t</div>\n\t<div id="c_box">\n\t  <div class="logo_co" style="position:absolute;"><img src="/img/13_법제처.jpg" /></div> <div class="c_msg2" style="position:absolute;"><br><br>\r\n안정적인 서비스 제공을 위한 시스템 점검으로 인하여<br>\r\n현재 웹서비스가 중단되고 있음을 알려드립니다.<br>\r\n더욱 편리한 서비스를 제공하기 위해 최선을 다하겠습니다.<br></div>\n\t</div>\n\t<div id="b_box">\n\t\t\n</div>\n</div>\n</body>\n</html>\n'

In [0]:
total_page_num, total_item_num = getTotalPageNum()

In [0]:
%%capture cap1
%%time

print(  '-'*50, '\n',
        'Start : ', datetime.today().strftime('%Y-%m-%d %H:%M:%S'), '\n',
        'Total Item Number : ', total_item_num, '\n',
        'Total Page Number : ', total_page_num, '\n',
        '-'*50, '\n',)

pages_completed = []
text_merged = ''

for page_num in range(1, total_page_num + 1):

    uri = createURIstring(uri_OC, uri_target, uri_type, page_num, uri_display, uri_sort)
    r = requests.get(uri)

    text_merged = text_merged + r.text
    print(f'%s : %04d-%04d pages saved.' % (uri, page_num, total_page_num))
    sleep( randint(5,10) )

print( '-'*50, '\n', )

In [0]:
# p = re.compile('<page>[0-9]+</page>')
# p.findall(text_merged)

In [0]:
# text_merged

In [0]:
# XML 파일을 통합하여 하나로 저장함.
with open(f'{directory}/{directory}_{today}.txt', 'w') as f:
    f.write(text_merged)
    f.close()

In [0]:
#(2/3) 법령 목록 Raw-Data 저장 파일을 변환 : XML string to JSON
f = open(f'./{directory}/{directory}_{today}.txt', 'r').read()
# f[:1000]

In [0]:
# p1 = re.compile(r'''<[?]xml version="1.0" encoding="UTF-8"[?]>''')
# p1.findall(f)

In [0]:
header = '<?xml version="1.0" encoding="UTF-8"?>'
f_replaced = f.replace(header, '')
f_replaced = header + '<root>' + f_replaced + '</root>' # Trouble shooting : https://www.codeproject.com/questions/1059071/error-parsing-xml-junk-after-document-element

# f_replaced[:1000]

In [0]:
with open(f'./{directory}/{directory}_{today}.json', 'w', encoding="utf-8") as outfile:
    json.dump(xmltodict.parse(f_replaced), outfile)

In [0]:
#(3/3) 법령 목록 JSON 파일을 기준으로 세부 법령 내용 요청

with open(f'./{directory}/{directory}_{today}.json', 'r', encoding="utf-8") as json_data:
    data = json.loads(json_data.read())

In [27]:
print(
    'Data Type : %s' % type(data), '\n',
    'Keys of dict : %s' % data.keys(), '\n',
)

Data Type : <class 'dict'> 
 Keys of dict : dict_keys(['root']) 



In [28]:
print(
    'Wrapped Pages DType : %s' % type( data['root']['LawSearch'] ), '\n',
    'Total Pages : %03d' % len( data['root']['LawSearch'] ), '\n',
)

Wrapped Pages DType : <class 'list'> 
 Total Pages : 048 



In [29]:
print(
    'Page Return Status', '-'*50, '\n',
    'Data Type of Each Page : %s' % type( data['root']['LawSearch'][0] ), '\n',
    'N-Keys of Each Page : %03d' % len( data['root']['LawSearch'][0] ), '\n',
    'Keys of Each Page : %s' % data['root']['LawSearch'][0].keys(), '\n'
)

Page Return Status -------------------------------------------------- 
 Data Type of Each Page : <class 'dict'> 
 N-Keys of Each Page : 009 
 Keys of Each Page : dict_keys(['target', '키워드', 'section', 'totalCnt', 'page', 'numOfRows', 'resultCode', 'resultMsg', 'law']) 



In [30]:
for key in data['root']['LawSearch'][0].keys():
    print(key, data['root']['LawSearch'][0][key])

target law
키워드 *
section lawNm
totalCnt 4754
page 1
numOfRows 100
resultCode 00
resultMsg success
law [{'@id': '1', '법령일련번호': '180715', '현행연혁코드': '현행', '법령명한글': '10ㆍ27법난 피해자의 명예회복 등에 관한 법률', '법령약칭명': '10ㆍ27법난법', '법령ID': '010719', '공포일자': '20160203', '공포번호': '13954', '제개정구분명': '일부개정', '소관부처코드': '1371000', '소관부처명': '문화체육관광부', '법령구분명': '법률', '시행일자': '20160630', '자법타법여부': None, '법령상세링크': '/DRF/lawService.do?OC=tooget&target=law&MST=180715&type=HTML&mobileYn=&efYd=20160630'}, {'@id': '2', '법령일련번호': '183817', '현행연혁코드': '현행', '법령명한글': '10ㆍ27법난 피해자의 명예회복 등에 관한 법률 시행령', '법령약칭명': '10ㆍ27법난법시행령', '법령ID': '010831', '공포일자': '20160608', '공포번호': '27211', '제개정구분명': '일부개정', '소관부처코드': '1371000', '소관부처명': '문화체육관광부', '법령구분명': '대통령령', '시행일자': '20160630', '자법타법여부': None, '법령상세링크': '/DRF/lawService.do?OC=tooget&target=law&MST=183817&type=HTML&mobileYn=&efYd=20160630'}, {'@id': '3', '법령일련번호': '200188', '현행연혁코드': '현행', '법령명한글': '119구조ㆍ구급에 관한 법률', '법령약칭명': '119법', '법령ID': '011349', '공포일자': '20171226', '공포번호': '1

In [31]:
data['root']['LawSearch'][0]['law']

[{'@id': '1',
  '공포번호': '13954',
  '공포일자': '20160203',
  '법령ID': '010719',
  '법령구분명': '법률',
  '법령명한글': '10ㆍ27법난 피해자의 명예회복 등에 관한 법률',
  '법령상세링크': '/DRF/lawService.do?OC=tooget&target=law&MST=180715&type=HTML&mobileYn=&efYd=20160630',
  '법령약칭명': '10ㆍ27법난법',
  '법령일련번호': '180715',
  '소관부처명': '문화체육관광부',
  '소관부처코드': '1371000',
  '시행일자': '20160630',
  '자법타법여부': None,
  '제개정구분명': '일부개정',
  '현행연혁코드': '현행'},
 {'@id': '2',
  '공포번호': '27211',
  '공포일자': '20160608',
  '법령ID': '010831',
  '법령구분명': '대통령령',
  '법령명한글': '10ㆍ27법난 피해자의 명예회복 등에 관한 법률 시행령',
  '법령상세링크': '/DRF/lawService.do?OC=tooget&target=law&MST=183817&type=HTML&mobileYn=&efYd=20160630',
  '법령약칭명': '10ㆍ27법난법시행령',
  '법령일련번호': '183817',
  '소관부처명': '문화체육관광부',
  '소관부처코드': '1371000',
  '시행일자': '20160630',
  '자법타법여부': None,
  '제개정구분명': '일부개정',
  '현행연혁코드': '현행'},
 {'@id': '3',
  '공포번호': '15298',
  '공포일자': '20171226',
  '법령ID': '011349',
  '법령구분명': '법률',
  '법령명한글': '119구조ㆍ구급에 관한 법률',
  '법령상세링크': '/DRF/lawService.do?OC=tooget&target=law&MST=200188&t

In [32]:
pd.DataFrame(
    data['root']['LawSearch'][0]['law'],
)

,@id,공포번호,공포일자,법령ID,법령구분명,법령명한글,법령상세링크,법령약칭명,법령일련번호,소관부처명,소관부처코드,시행일자,자법타법여부,제개정구분명,현행연혁코드
0,1,13954,20160203,010719,법률,10ㆍ27법난 피해자의 명예회복 등에 관한 법률,/DRF/lawService.do?OC=tooget&target=law&MST=18...,10ㆍ27법난법,180715,문화체육관광부,1371000,20160630,None,일부개정,현행
1,2,27211,20160608,010831,대통령령,10ㆍ27법난 피해자의 명예회복 등에 관한 법률 시행령,/DRF/lawService.do?OC=tooget&target=law&MST=18...,10ㆍ27법난법시행령,183817,문화체육관광부,1371000,20160630,None,일부개정,현행
2,3,15298,20171226,011349,법률,119구조ㆍ구급에 관한 법률,/DRF/lawService.do?OC=tooget&target=law&MST=20...,119법,200188,소방청,1661000,20180627,None,일부개정,현행
3,4,28216,20170726,011452,대통령령,119구조ㆍ구급에 관한 법률 시행령,/DRF/lawService.do?OC=tooget&target=law&MST=19...,119법시행령,195947,소방청,1661000,20170726,None,타법개정,현행
4,5,00002,20170726,011457,행정안전부령,119구조ㆍ구급에 관한 법률 시행규칙,/DRF/lawService.do?OC=tooget&target=law&MST=19...,119법시행규칙,196225,소방청,1661000,20170726,None,타법개정,현행
5,6,04401,19691204,004546,대통령령,1945년이후종전의규정에의한학교졸업자자격인정령,/DRF/lawService.do?OC=tooget&target=law&MST=24...,None,24144,교육부,1342000,19691204,None,전부개정,현행
6,7,10820,20110714,009678,법률,1959년 12월 31일 이전에 퇴직한 군인의 퇴직급여금지급에 관한 특별법,/DRF/lawService.do?OC=tooget&target=law&MST=11...,1959이전군퇴직금법,114766,국방부,1290000,20120101,None,일부개정,현행
7,8,27083,20160405,009990,대통령령,1959년 12월 31일 이전에 퇴직한 군인의 퇴직급여금지급에 관한 특별법 시행령,/DRF/lawService.do?OC=tooget&target=law&MST=18...,1959이전군퇴직금법시행령,182269,국방부,1290000,20160405,None,타법개정,현행
8,9,04101,19890329,001348,법률,1980년해직공무원의보상등에관한특별조치법,/DRF/lawService.do?OC=tooget&target=law&MST=32...,1980해직자보상법,3274,인사혁신처,1760000,19890329,None,제정,현행
9,10,29181,20180918,004549,대통령령,1980년해직공무원의보상등에관한특별조치법시행령,/DRF/lawService.do?OC=tooget&target=law&MST=20...,1980해직자보상법시행령,204611,인사혁신처,1760000,20180921,None,타법개정,현행


In [0]:
%%capture cap2
%%time

df_list_of_laws = pd.DataFrame()

for page_idx in range(0, len(data['root']['LawSearch'])):
    
    df_list_of_laws = pd.concat([
                            df_list_of_laws,
                            pd.DataFrame( data['root']['LawSearch'][page_idx]['law'] ),
                      ])
    
    print('Return %s, Dataframe of Page Number %s (%s) concatenated' % (
                                                        data['root']['LawSearch'][page_idx]['resultMsg'],
                                                        data['root']['LawSearch'][page_idx]['page'],
                                                        data['root']['LawSearch'][page_idx]['totalCnt'],)
    )

In [0]:
df_list_of_laws.to_csv(f'./{directory}/{directory}_{today}.csv', encoding='utf-8', index=False)

In [0]:
# 조회 내용 로그를 저장함.
with open(f'{directory}/{directory}_{today}_result.txt', 'w') as f:
    f.write(cap1.stdout)
    f.write(cap2.stdout)
    f.close()

In [36]:
df_list_of_laws['법령구분명'].unique()

array(['법률', '대통령령', '행정안전부령', '총리령', '대법원규칙', '환경부령', '여성가족부령',
       '농림축산식품부령', '중앙선거관리위원회규칙', '국토교통부령', '보건복지부령', '감사원규칙', '기획재정부령',
       '통일부령', '건설교통부령', '고용노동부령', '법무부령', '문화체육관광부령', '헌법재판소규칙',
       '산업통상자원부령', '행정자치부령', '교육부령', '국방부령', '재정경제부령', '국회규칙', '지식경제부령',
       '해양수산부령', '과학기술정보통신부령', '교육인적자원부령', '국토해양부령', '농림수산식품부령', '외교부령',
       '교육과학기술부령', '노동부령', '중소벤처기업부령', '헌법', '보건복지가족부령', '상공부령', '경제기획원령',
       '대통령긴급명령', '각령'], dtype=object)

In [37]:
# list_of_industry_law = df_list_of_laws[ (df_list_of_laws['법령명한글'].str.contains('금융')) & (df_list_of_laws['법령구분명'].str.contains('법')) ].reset_index(drop=True)
list_of_industry_law = df_list_of_laws[ (df_list_of_laws['법령구분명'].str.contains('법률')) | (df_list_of_laws['법령구분명'].str.contains('헌법')) ].reset_index(drop=True)
list_of_industry_law

,@id,공포번호,공포일자,법령ID,법령구분명,법령명한글,법령상세링크,법령약칭명,법령일련번호,소관부처명,소관부처코드,시행일자,자법타법여부,제개정구분명,현행연혁코드
0,1,13954,20160203,010719,법률,10ㆍ27법난 피해자의 명예회복 등에 관한 법률,/DRF/lawService.do?OC=tooget&target=law&MST=18...,10ㆍ27법난법,180715,문화체육관광부,1371000,20160630,None,일부개정,현행
1,3,15298,20171226,011349,법률,119구조ㆍ구급에 관한 법률,/DRF/lawService.do?OC=tooget&target=law&MST=20...,119법,200188,소방청,1661000,20180627,None,일부개정,현행
2,7,10820,20110714,009678,법률,1959년 12월 31일 이전에 퇴직한 군인의 퇴직급여금지급에 관한 특별법,/DRF/lawService.do?OC=tooget&target=law&MST=11...,1959이전군퇴직금법,114766,국방부,1290000,20120101,None,일부개정,현행
3,9,04101,19890329,001348,법률,1980년해직공무원의보상등에관한특별조치법,/DRF/lawService.do?OC=tooget&target=law&MST=32...,1980해직자보상법,3274,인사혁신처,1760000,19890329,None,제정,현행
4,11,14839,20170726,011370,법률,1인 창조기업 육성에 관한 법률,/DRF/lawService.do?OC=tooget&target=law&MST=19...,1인창조기업법,195172,중소벤처기업부,1421000,20170726,None,타법개정,현행
5,13,14912,20171024,011532,법률,2018 평창 동계올림픽대회 및 동계패럴림픽대회 지원 등에 관한 특별법,/DRF/lawService.do?OC=tooget&target=law&MST=19...,평창올림픽법,198231,문화체육관광부,1371000,20181025,None,타법개정,현행
6,16,12843,20141119,012159,법률,4ㆍ16세월호참사 진상규명 및 안전사회 건설 등을 위한 특별법,/DRF/lawService.do?OC=tooget&target=law&MST=16...,세월호진상규명법,162682,해양수산부,1192000,20150101,None,제정,현행
7,18,15461,20180313,012245,법률,4ㆍ16세월호참사 피해구제 및 지원 등을 위한 특별법,/DRF/lawService.do?OC=tooget&target=law&MST=20...,세월호피해지원법,202681,"국무조정실,해양수산부",1092000 1192000,20180614,None,일부개정,현행
8,21,15473,20180313,009289,법률,5ㆍ18민주유공자예우에 관한 법률,/DRF/lawService.do?OC=tooget&target=law&MST=20...,5ㆍ18유공자법,202693,국가보훈처,1180000,20180614,None,일부개정,현행
9,24,12910,20141230,001234,법률,5ㆍ18민주화운동 관련자 보상 등에 관한 법률,/DRF/lawService.do?OC=tooget&target=law&MST=16...,5ㆍ18보상법,165518,행정안전부,1741000,20141230,None,일부개정,현행


In [0]:
list_of_industry_law['법령명한글_길이'] = list_of_industry_law['법령명한글'].apply( lambda x: len(x) )

In [39]:
list_of_industry_law[ list_of_industry_law['법령명한글_길이'] > 40 ]

,@id,공포번호,공포일자,법령ID,법령구분명,법령명한글,법령상세링크,법령약칭명,법령일련번호,소관부처명,소관부처코드,시행일자,자법타법여부,제개정구분명,현행연혁코드,법령명한글_길이
2,7,10820,20110714,009678,법률,1959년 12월 31일 이전에 퇴직한 군인의 퇴직급여금지급에 관한 특별법,/DRF/lawService.do?OC=tooget&target=law&MST=11...,1959이전군퇴직금법,114766,국방부,1290000,20120101,None,일부개정,현행,41
140,486,14116,20160329,010588,법률,공중 등 협박목적 및 대량살상무기확산을 위한 자금조달행위의 금지에 관한 법률,/DRF/lawService.do?OC=tooget&target=law&MST=18...,테러자금금지법,182070,금융위원회,1160100,20170330,None,타법개정,현행,42
197,707,09401,20090130,001417,법률,국가보위에 관한 특별조치법 제5조제4항에 의한 동원대상지역 내의 토지의 수용ㆍ사용에...,/DRF/lawService.do?OC=tooget&target=law&MST=91...,국보위수용토지법,91389,국방부,1290000,20090731,None,타법개정,현행,84
430,1497,14839,20170726,011169,법률,대일항쟁기 강제동원 피해조사 및 국외강제동원 희생자 등 지원에 관한 특별법,/DRF/lawService.do?OC=tooget&target=law&MST=19...,강제동원조사법,195071,행정안전부,1741000,20170726,None,타법개정,현행,41
441,1533,09360,20090130,001241,법률,대한민국과 아메리카합중국 간의 상호방위조약 제4조에 의한 시설과 구역 및 대한민국에...,/DRF/lawService.do?OC=tooget&target=law&MST=91...,주한미군민사법,91501,법무부,1270000,20090130,None,일부개정,현행,80
442,1536,10583,20110414,001116,법률,대한민국과 아메리카합중국 간의 상호방위조약 제4조에 의한 시설과 구역 및 대한민국에...,/DRF/lawService.do?OC=tooget&target=law&MST=11...,주한미군형사법,112382,법무부,1270000,20110414,None,일부개정,현행,80
443,1537,10825,20110714,000929,법률,대한민국과 아메리카합중국 간의 상호방위조약 제4조에 의한 시설과 구역 및 대한민국에...,/DRF/lawService.do?OC=tooget&target=law&MST=11...,미군공여재산법,114769,국방부,1290000,20110714,None,일부개정,현행,105
444,1538,13550,20151215,000593,법률,대한민국과 아메리카합중국 간의 상호방위조약 제4조에 의한 시설과 구역 및 대한민국에...,/DRF/lawService.do?OC=tooget&target=law&MST=17...,주한미군관세법,177196,기획재정부,1051000,20151231,None,타법개정,현행,92
496,1674,15622,20180529,013128,법률,드루킹의 인터넷상 불법 댓글 조작 사건과 관련된 진상규명을 위한 특별검사의 임명 등...,/DRF/lawService.do?OC=tooget&target=law&MST=20...,None,203628,법무부,1270000,20180529,None,제정,현행,53
551,1846,14276,20161122,012675,법률,박근혜 정부의 최순실 등 민간인에 의한 국정농단 의혹 사건 규명을 위한 특별검사의 ...,/DRF/lawService.do?OC=tooget&target=law&MST=18...,None,187653,법무부,1270000,20161122,None,제정,현행,57


In [0]:
def createURIwithLawIdquery(OC, target, type, lawID):
    uri = f'http://www.law.go.kr/DRF/lawService.do?OC={OC}&target={target}&type={type}&ID={lawID}'
    print(uri, '\n')
    return uri

In [0]:
def saveEachLawContent(lawName, lawID):
    uri = createURIwithLawIdquery( uri_OC, uri_target, uri_type, lawID )

    r = requests.get(uri)
    text = r.text
    
    if len(lawName) > 40:
        lawName = lawName[:20]   # Trouble Shooting : OSError: [Errno 36] File name too long: '01_rawdata/01_rawdata_000929_대한민국과 아메리카합중국 간의 상호방위조약 제4조에 의한 시설과 구역 및 대한민국에서의 합중국 군대의 지위에 관한 협정의 시행에 따른 국가 및 지방자치단체의 재산의 관리와 처분에 관한 법률_20181102.txt'
    
    # XML 파일을 저장함.
    with open(f'{directory}/{directory}_{lawID}_{lawName}_{today}.txt', 'w') as f:
        f.write(text)
        f.close()
    print(f'{directory}/{directory}_{lawID}_{lawName}_{today}.txt', ' saved.')
    
    # 법령 목록 Raw-Data 저장 파일을 변환 : XML string to JSON
    f = open(f'./{directory}/{directory}_{lawID}_{lawName}_{today}.txt', 'r').read()

    header = '<?xml version="1.0" encoding="UTF-8"?>'
    f_replaced = f.replace(header, '')
    f_replaced = header + '<root>' + f_replaced + '</root>' # Trouble shooting : https://www.codeproject.com/questions/1059071/error-parsing-xml-junk-after-document-element

    with open(f'./{directory}/{directory}_{lawID}_{lawName}_{today}.json', 'w', encoding="utf-8") as outfile:
        json.dump(xmltodict.parse(f_replaced), outfile)
    print(f'{directory}/{directory}_{lawID}_{lawName}_{today}.json', ' saved.')

In [0]:
%%capture cap3
%%time

for i in range(len(list_of_industry_law)):

    
    saveEachLawContent(
        str( list_of_industry_law.loc[i]['법령명한글'] ),
        str( list_of_industry_law.loc[i]['법령ID'] ),
    )

In [0]:
def readJSONtoDataframe(json_file_path):

    #법령 JSON 파일을 기준으로 세부 법령 내용 요청
    with open(json_file_path, 'r', encoding="utf-8") as json_data:
        data = json.loads(json_data.read())
    
    return data

In [0]:
# 조회 내용 로그를 저장함.
with open(f'{directory}/{directory}_saveEachLawContent_{today}_result.txt', 'w') as f:
    f.write(cap3.stdout)
    f.close()

In [0]:
json_010513 = readJSONtoDataframe( f'./{directory}/{directory}_010513_자본시장과 금융투자업에 관한 법률_{today}.json' )

In [47]:
print(
    'Data Type : %s' % type(json_010513), '\n',
    'Keys of dict : %s' % json_010513.keys(), '\n',
)

Data Type : <class 'dict'> 
 Keys of dict : dict_keys(['root']) 



In [48]:
print(
    'Data Type : %s' % type(json_010513['root']), '\n',
    'Keys of dict : %s' % json_010513['root'].keys(), '\n',
)

Data Type : <class 'dict'> 
 Keys of dict : dict_keys(['법령']) 



In [49]:
print(
    'Data Type : %s' % type(json_010513['root']['법령']), '\n',
    'Keys of dict : %s' % json_010513['root']['법령'].keys(), '\n',
)

Data Type : <class 'dict'> 
 Keys of dict : dict_keys(['@법령키', '기본정보', '조문', '부칙', '별표', '개정문', '제개정이유']) 



In [0]:
contents_list = []

def extractJsonContent(docu):
    if type(docu) == list:
        for element in docu:
            extractJsonContent(element)
    elif type(docu) == dict:
        key_list = list(docu.keys())
        for key in key_list:
            extractJsonContent(docu[key])
    else:
        contents_list.append(docu)

In [0]:
extractJsonContent(json_010513['root']['법령'])

In [57]:
!ls ./01_rawdata/ -al

total 405752
drwxr-xr-x 2 root root  319488 Nov  2 11:59  .
drwxr-xr-x 1 root root    4096 Nov  2 11:10  ..
-rw-r--r-- 1 root root   27780 Nov  2 11:46  01_rawdata_000001_재외국민등록법_20181102.json
-rw-r--r-- 1 root root   21605 Nov  2 11:46  01_rawdata_000001_재외국민등록법_20181102.txt
-rw-r--r-- 1 root root   20372 Nov  2 11:46 '01_rawdata_000002_재외공관용 재산의 취득ㆍ관리 등에 관한 특례법_20181102.json'
-rw-r--r-- 1 root root   14754 Nov  2 11:46 '01_rawdata_000002_재외공관용 재산의 취득ㆍ관리 등에 관한 특례법_20181102.txt'
-rw-r--r-- 1 root root   16975 Nov  2 11:46 '01_rawdata_000003_재외공관 수입금 등 직접사용에 관한 법률_20181102.json'
-rw-r--r-- 1 root root   12544 Nov  2 11:46 '01_rawdata_000003_재외공관 수입금 등 직접사용에 관한 법률_20181102.txt'
-rw-r--r-- 1 root root   99289 Nov  2 11:46 '01_rawdata_000004_재외공관 공증법_20181102.json'
-rw-r--r-- 1 root root   76947 Nov  2 11:46 '01_rawdata_000004_재외공관 공증법_20181102.txt'
-rw-r--r-- 1 root root   73557 Nov  2 11:42  01_rawdata_000006_여권법_20181102.json
-rw-r--r-- 1 root root   54412 Nov  2 11:42  01_rawdata_00000

In [58]:
# google.colab 의 drive 라이브러리 이용
from google.colab import drive
drive.mount('/content/gdrive')

# !ls ./gdrive/M* -al

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [63]:
import os

cwd = os.getcwd()
print('Current Working Directory:', cwd)

gdrive = os.path.join(os.getcwd(), 'gdrive', 'My Drive')
print('Check Connection with Google Drive:', gdrive)

dataPath = os.path.join(gdrive, '세미나+스터디')
print('Move to 3rdCompetitionDataPath:', dataPath)

Current Working Directory: /content
Check Connection with Google Drive: /content/gdrive/My Drive
Move to 3rdCompetitionDataPath: /content/gdrive/My Drive/세미나+스터디


In [60]:
!ls -al

total 336
drwxr-xr-x 1 root root   4096 Nov  2 12:02 .
drwxr-xr-x 1 root root   4096 Nov  2 11:10 ..
drwxr-xr-x 2 root root 319488 Nov  2 11:59 01_rawdata
drwxr-xr-x 4 root root   4096 Oct 31 16:34 .config
drwx------ 3 root root   4096 Nov  2 12:02 gdrive
drwxr-xr-x 2 root root   4096 Oct 31 16:48 sample_data


In [65]:
!ls -al gdrive/My*/세미나+스터디

total 731
drwx------ 2 root root   4096 Nov  2 12:01  01_rawdata
-rw------- 1 root root 715406 Nov  2 12:03  01_현행법령목록_v0.1.ipynb
drwx------ 2 root root   4096 Oct  2 10:56 '[201801]T아카데미 도커'
drwx------ 2 root root   4096 Oct  2 10:58 '[201802]alphalaw'
drwx------ 2 root root   4096 Oct 27 00:35 '[201810] 머신러닝 인 파이썬(도서)'
drwx------ 2 root root   4096 Oct  2 10:59 '[201810]남명학사 명사초청 - 김태유'
drwx------ 2 root root   4096 Oct 25 09:11 '[201810]빅데이터를 활용한 부동산 시장'
drwx------ 2 root root   4096 Oct 14 10:23 '[201810]alphalaw'
drwx------ 2 root root   4096 Oct 25 09:10 '[201810]공개SW'


In [0]:
!cp -r 01_rawdata gdrive/My*/세미나+스터디/

In [71]:
!ls -R gdrive/My*/세미나+스터디/01_rawdata | wc -l

2975


In [72]:
!ls -R 01_rawdata | wc -l

2975
